In [1]:
#from nipype.interfaces import niftyreg
import numpy as np
from skimage import io
from pathlib import Path
import re
import ants
from skimage.transform import resize
from tqdm import tqdm
from skimage.morphology import skeletonize_3d, binary_dilation, binary_closing
from scipy.ndimage import distance_transform_edt
import tifffile as tif
from scipy.ndimage import binary_fill_holes
import cc3d
from scipy.io import loadmat, savemat
import skan
import sknw
import networkx as nx
import pickle
import os
import matplotlib.pyplot as plt
import pandas as pd
import time

# Define connected componnet removal

In [2]:
def remove_small_comps_3d(image, thresh = 500):
    """
    

    Parameters
    ----------
    image : binary np array with uint8 elements
        3d numpy matrix, connected components will be removed form this image
    thresh : int64
        smallest connected components to keep

    Returns
    -------
    np.array with uint8 elements, binary
        binary image with connected components below the threshold removed.

    """
    img_lab, N = cc3d.connected_components(image,return_N=True)
    unique, counts = np.unique(img_lab, return_counts=True)
    unique_keep = unique[counts>thresh]
    unique_keep = np.delete(unique_keep,[0])
    img_filt = np.zeros(img_lab.shape).astype('int8')
    img_filt[np.isin(img_lab,unique_keep)] = 1
    return img_filt.astype('uint8')   

def fill_holes(img,thresh=100):
    #res = np.zeros(img.shape)
    for i in np.unique(img)[::-1]:
        _tmp = (img==i)*1.0
        _tmp = _tmp.astype('int8')
        _tmp = remove_small_comps_3d(_tmp,thresh=thresh)
        img[_tmp==1] = i
    res = img.astype('int8')
    return res

# Get mean predictions

In [3]:
directory = Path('matt_preds')
files  = directory.glob('*-*_mean.npy')
files = sorted([x.as_posix() for x in files])
#files = [x for x in files if 'vbm' not in x]

In [4]:
len(files)

767

# High bias low varience segmentation

With removal of connected components under 500 pixels

In [7]:
min_prob = 0.75
max_var = 0.1
for file in tqdm(files[::-1]):
    if (time.time() - os.path.getmtime(re.sub('mean','seg',file)))/3600>48:
    #if not os.path.exists(re.sub('mean','seg',file)):
        mean = np.load(file)
        std = np.load(re.sub('mean','std',file))
        seg = np.zeros(mean.shape[1:])
        seg[(mean[1,:,:,:] > min_prob) * (std[1,:,:,:] < max_var)] = 1
        seg[(mean[2,:,:,:] > min_prob) * (std[2,:,:,:] < max_var)] = 2
        seg = seg.astype('int8')
        seg = (seg==1)*1
        seg = fill_holes(seg)
        np.save(re.sub('mean','seg',file),seg)
        #savemat(re.sub('mean.npy','seg.mat',file),{'FinalImage':fill_holes(binary_dilation(binary_dilation(seg)))})
        #tif.imwrite(re.sub('mean.npy','seg.tif',file),seg)

100%|██████████| 767/767 [38:39<00:00,  3.02s/it]  


# Get distance transform of neuron segmentation

In [5]:
directory = Path('matt_preds')
files  = directory.glob('*-*_mean.npy')
files = sorted([x.as_posix() for x in files])
#files = [x for x in files if 'vbm' not in x]

In [6]:
min_prob = 0.75
max_var = 0.1
for file in tqdm(files[::-1]):
    if (time.time() - os.path.getmtime(re.sub('mean','seg_nrn_dst',file)))/3600>48:
    #if not os.path.exists(re.sub('mean','seg_nrn_dst',file)):
        mean = np.load(file)
        std = np.load(re.sub('mean','std',file))
        seg = np.zeros(mean.shape[1:])
        seg[(mean[1,:,:,:] > min_prob) * (std[1,:,:,:] < max_var)] = 1
        seg[(mean[2,:,:,:] > min_prob) * (std[2,:,:,:] < max_var)] = 2
        seg = seg.astype('int8')
        seg = (seg==2)*1
        np.save(re.sub('mean','seg_nrn',file),seg)
        np.save(re.sub('mean','seg_nrn_dst',file),distance_transform_edt(1-seg))

100%|██████████| 767/767 [00:00<00:00, 1912.07it/s]


In [7]:
re.sub('mean','seg_nrn_dst',file)

'matt_preds/06162021_45-XYZres288_0001_seg_nrn_dst.npy'

# ANTs registration

## Upsample raw images

In [9]:
image_path = Path('/home/rozakmat/projects/rrg-bojana/data/THY1-TBI')
images = list(image_path.glob('*?[0-9]/*res*?[0-9].tif'))
images = sorted([x.as_posix() for x in images])
len(images)

768

In [10]:
for i in tqdm(range(len(images))[:]):
    if not os.path.exists('matt_preds/'+re.sub('/','-',re.sub('/home/rozakmat/projects/rrg-bojana/data/THY1-TBI/','',images[i]))):
        image = io.imread(images[i])
        if image.shape[0] == 96:
            image = np.swapaxes(image,0,1)
            image = np.swapaxes(image,1,3)
            image = resize(image,(2,507,507,252),preserve_range=True, order=3)
            zeros = np.zeros((1,507,507,252)).astype('uint16')
            image = np.append(image,zeros,axis = 0)
            image = image.astype('float16')
            io.imsave('matt_preds/'+re.sub('/','-',re.sub('/home/rozakmat/projects/rrg-bojana/data/THY1-TBI/','',images[i])),image)

100%|██████████| 768/768 [00:44<00:00, 17.10it/s]


In [11]:
'matt_preds/'+re.sub('/','-',re.sub('/home/rozakmat/projects/rrg-bojana/data/THY1-TBI/','',images[i]))

'matt_preds/vbm11 Apr 04 2020-XYZres052_0001.tif'

## reshape numpy seg files and resave as tif

In [12]:
directory_seg = Path('matt_preds')
files_seg  = list(directory_seg.glob('*_seg.npy'))
files_seg = sorted([x.as_posix() for x in files_seg])
#files_seg

In [13]:
for i in tqdm(range(len(files_seg))):
    if not os.path.exists(re.sub('.npy','.tif',files_seg[i])):
        tmp = np.load(files_seg[i])
        tmp = np.reshape(tmp,(1,507,507,252))
        io.imsave(re.sub('.npy','.tif',files_seg[i]),tmp)

100%|██████████| 1157/1157 [00:26<00:00, 44.44it/s]


## Register images and transform masks

In [14]:
directory_seg = Path('matt_preds')
images = list(directory_seg.glob('*_0001.tif'))
images = sorted([x.as_posix() for x in images])#[0:2]
print(len(images))
print(images[0])

380
matt_preds/06162021_45-XYZres288_0001.tif


In [15]:
for i in tqdm(range(len(images))):
    if os.path.exists(re.sub('.tif','_seg.tif',images[i])):
        if not os.path.exists(re.sub('seg','seg_warped',re.sub('.tif','_seg.tif',images[i]))):
            fix = ants.image_read(re.sub('_0001','',images[i]))
            mov = ants.image_read(images[i])
            fix_mask = ants.image_read(re.sub('.tif','_seg.tif',re.sub('_0001','',images[i])))
            mov_mask = ants.image_read(re.sub('.tif','_seg.tif',images[i]))
            mytx = ants.registration(fixed = fix,
                                    moving = mov,
                                    type_of_transform = 'Rigid'
                                    )
            warpedmask = ants.apply_transforms(fixed = fix_mask,
                                               moving = mov_mask,
                                               transformlist = mytx['fwdtransforms'],
                                               interpolator = 'nearestNeighbor'
                                              )
            io.imsave(re.sub('seg','seg_warped',re.sub('.tif','_seg.tif',images[i])),warpedmask.numpy())
            io.imsave(re.sub('seg','seg_warped',re.sub('.tif','_seg.tif',re.sub('_0001','',images[i]))),fix_mask.numpy())
            #print(2 * np.sum(warpedmask.numpy()*fix_mask.numpy())/(np.sum(warpedmask.numpy())+np.sum(fix_mask.numpy())))

100%|██████████| 380/380 [00:10<00:00, 36.52it/s]


## Save Matlab .mat file of registered images

In [16]:
directory_seg = Path('matt_preds')
images = list(directory_seg.glob('*_0001_seg_warped.tif'))
images = sorted([x.as_posix() for x in images])[::-1]

In [17]:
for image in tqdm(images):
    if not os.path.exists(re.sub('_seg_warped.tif','_seg_warped_single.mat',re.sub('_0001','',image))):
        img_0001 = io.imread(image)
        img = io.imread(re.sub('_0001','',image))
        seg = img*img_0001
        seg = (seg==1)*1
        seg = seg.astype('int8')
        seg = fill_holes(seg)
        savemat(re.sub('_seg_warped.tif','_seg_warped_single.mat',re.sub('_0001','',image)),{'FinalImage':fill_holes(binary_dilation(seg))})
        
        

100%|██████████| 575/575 [00:15<00:00, 36.60it/s]


In [18]:
print(image)

matt_preds/06162021_45-XYZres288_0001_seg_warped.tif


# Generate Graphs

In [20]:
directory = Path('matt_preds')
files_seg_0001 = directory.glob('*_0001_seg_warped.tif')
files_seg_0001 = sorted([x.as_posix() for x in files_seg_0001])
len(files_seg_0001)

575

In [21]:
for file_0001 in tqdm(files_seg_0001):
    if not os.path.exists(re.sub('_0001_seg_warped.tif','_warped.pickle',file)):
        file = file_0001
        skel_file = re.sub('_0001_seg_warped.tif','_skel_warped_single.mat',file)
        #seg = io.imread(file)
        skel = loadmat(skel_file)['FilteredImage']
        #skel = skeletonize_3d(skel)
        io.imsave(re.sub('_seg_warped.tif','_single_skel.tif',file),skel)
        #dst_tsfm = distance_transform_edt(seg)
        #dst_tsfm[dst_tsfm==0]=0.00001
        #skel_dst = skel*dst_tsfm
    #
        #
        #np.save(re.sub('seg','dst_skel',file),skel_dst)
        #io.imsave(re.sub('_seg_warped.tif','_dst_skel_warped.tif',file),skel_dst)
        graph = sknw.build_sknw(skel, multi=False)
        print(len(graph.edges))
        #graph, c0  = skan.csr.skeleton_to_csgraph(skel)
        #print(len(graph.edges))
        
        #print(len(graph_0001.edges))
        #pickle.dump(graph, open(str(re.sub('_seg.npy','.pickle',file)), 'w'))
        #nx.write_pajek(graph,re.sub('_seg.npy','.pajek',file))
        
        nx.write_gpickle(graph,re.sub('_0001_seg_warped.tif','_warped.pickle',file))

100%|██████████| 575/575 [00:00<00:00, 9893.57it/s]


# write vessel measurments to graph files

In [28]:
directory = Path('matt_preds')
files = directory.glob('*_warped.pickle')
files = sorted([x.as_posix() for x in files])
files = [x for x in files if '-' in x]
print(len(files))
print(files[0])


380
matt_preds/06162021_45-XYZres288_warped.pickle


In [29]:
#re.sub('matt_preds/','',re.sub('_warped.pickle','',file)).split('-')[0].split('_')[1]

In [30]:
#re.sub('matt_preds/','',re.sub('_warped.pickle','',file))

In [38]:
i=0
xls = pd.ExcelFile('TBI_STIM_metalog_local.xlsx')
df = {}
for sheet_name in xls.sheet_names:
    df[sheet_name] = xls.parse(sheet_name)

for file in tqdm(files):
    #if not os.path.exists(re.sub('preds','preds_graphs',re.sub('.pickle','_radii.pickle',file))):
        graph = nx.read_gpickle(file)
        if len(graph.edges) < 1500:
            for sheet_name in xls.sheet_names:
                if re.sub('matt_preds/','',re.sub('_warped.pickle','',file)).split('-')[1] in df[sheet_name].values:
                    subj = sheet_name
                    if re.sub('matt_preds/','',re.sub('_warped.pickle','',file)).split('-')[0].split('_')[1] in sheet_name:
                        if subj in ["TBI07_3D",
                                    "TBI11_3D",
                                    "TBI22_3D",
                                    "TBI31_3D",
                                    "TBI38_3D",
                                    "SHAM09_3D",
                                    "SHAM12_3D",
                                    "SHAM23_3D",
                                    "SHAM32_3D",
                                    'TBI38_3D',
                                    'TBI6_3D',
                                    'SHAM7_3D',
                                    'TBI43_3D',
                                    'TBI45_3D',
                                    'SHAM53_3D',
                                    'SHAM56_3D',
                                    'SHAM55_3D']:
                            gender = 'male'
                        else:
                            gender = 'female'
                        treatment = re.sub('SHA','SHAM',subj[0:3])
                        _tmp = df[subj].loc[df[subj]['CHECK WATER'] == re.sub('matt_preds/','',re.sub('_warped.pickle','',file)).split('-')[1]]
                        if _tmp['Unnamed: 12'].iloc[0] == 'raster':
                            seg_file = re.sub('_warped.pickle','_seg_warped.tif',file)
                            seg_0001_file = re.sub('_warped.pickle','_0001_seg_warped.tif',file)
                            seg = io.imread(seg_file)
                            seg_0001 = io.imread(seg_0001_file)
                            seg_dst = distance_transform_edt(seg)
                            seg_0001_dst = distance_transform_edt(seg_0001)
                            nrn_dst = np.load(re.sub('_warped.pickle','_seg_nrn_dst.npy',file))
                            nrn_dst = np.swapaxes(nrn_dst,0,2)
                            wavelength = _tmp['Unnamed: 11'].iloc[0]
                            power_per = _tmp['Unnamed: 10'].iloc[0]
                            start_depth = _tmp['Unnamed: 2'].iloc[0]
                            age = _tmp['Unnamed: 14'].iloc[0]
                            days_post_injury = _tmp['Unnamed: 15'].iloc[0]
                            for i in range(len(graph.edges)):
                                path = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['pts']
                                _vals = seg_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                                _vals_0001 = seg_0001_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                                _nrn_dst_vals = nrn_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii'] = np.mean(_vals)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_std'] = np.std(_vals)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_0001'] = np.mean(_vals_0001)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_0001_std'] = np.std(_vals_0001)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['mean_neuron_distance'] = np.mean(_nrn_dst_vals)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['neuron_distance_std'] = np.std(_nrn_dst_vals)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['neuron_distance_min'] = np.min(_nrn_dst_vals)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['delta'] = np.mean(_vals_0001) - np.mean(_vals)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['gender'] = gender
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights'] = _vals
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights_0001'] = _vals_0001
                                #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight'] = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight']
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0z'] = path[0][0]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0y'] = path[0][1]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0x'] = path[0][2]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1z'] = path[-1][0]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1y'] = path[-1][1]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1x'] = path[-1][2]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['img_start_depth'] = start_depth
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['mean_depth'] = np.mean(path[:,0])
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['euclidean-dst'] = np.sqrt(np.sum(np.square(path[-1]-path[0])))
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['subject'] = subj
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['treatment'] = treatment
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['wavelength'] = wavelength
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['power'] = power_per
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['age'] = age
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['days_post_injury'] = days_post_injury
                            nx.write_gpickle(graph, re.sub('preds','preds_graphs',re.sub('.pickle','_radii.pickle',file)))
                            i+=1
print(i)

  0%|          | 0/380 [00:00<?, ?it/s]/tmp/ipykernel_1975335/87323968.py:76: RuntimeWarning:

invalid value encountered in sqrt

100%|██████████| 380/380 [1:50:33<00:00, 17.46s/it]  

404


In [39]:
print(nrn_dst.shape)
print(seg_dst.shape)

(252, 507, 507)
(252, 507, 507)


In [40]:
graphs = [file for file in files if not os.path.exists(re.sub('preds','preds_graphs',re.sub('.pickle','_radii.pickle',file)))]
print(len(graphs))

119


In [41]:
i=0
xls = pd.ExcelFile('TBI_STIM_metalog_local.xlsx')
df = {}
for sheet_name in xls.sheet_names:
    df[sheet_name] = xls.parse(sheet_name)

for file in tqdm(graphs):
    #if not os.path.exists(re.sub('preds','preds_graphs',re.sub('.pickle','_radii.pickle',file))):
        graph = nx.read_gpickle(file)
        if len(graph.edges) < 1500:
            for sheet_name in xls.sheet_names:
                if re.sub('matt_preds/','',re.sub('_warped.pickle','',file)).split('-')[1] in df[sheet_name].values:
                    subj = sheet_name
                    if re.sub('matt_preds/','',re.sub('_warped.pickle','',file)).split('-')[0].split('_')[1] in sheet_name:
                        if subj in ["TBI07_3D",
                                    "TBI11_3D",
                                    "TBI22_3D",
                                    "TBI31_3D",
                                    "TBI38_3D",
                                    "SHAM09_3D",
                                    "SHAM12_3D",
                                    "SHAM23_3D",
                                    "SHAM32_3D",
                                    'TBI38_3D',
                                    'TBI6_3D',
                                    'SHAM7_3D',
                                    'TBI43_3D',
                                    'TBI45_3D',
                                    'SHAM53_3D',
                                    'SHAM56_3D',
                                    'SHAM55_3D']:
                            gender = 'male'
                        else:
                            gender = 'female'
                        treatment = re.sub('SHA','SHAM',subj[0:3])
                        _tmp = df[subj].loc[df[subj]['CHECK WATER'] == re.sub('matt_preds/','',re.sub('_warped.pickle','',file)).split('-')[1]]
                        if _tmp['Unnamed: 10'].iloc[0] == 2:
                            seg_file = re.sub('_warped.pickle','_seg_warped.tif',file)
                            seg_0001_file = re.sub('_warped.pickle','_0001_seg_warped.tif',file)
                            seg = io.imread(seg_file)
                            seg_0001 = io.imread(seg_0001_file)
                            seg_dst = distance_transform_edt(seg)
                            seg_0001_dst = distance_transform_edt(seg_0001)
                            nrn_dst = np.load(re.sub('_warped.pickle','_seg_nrn_dst.npy',file))
                            nrn_dst = np.swapaxes(nrn_dst,0,2)
                            start_depth = _tmp['Unnamed: 2'].iloc[0]
                            age = _tmp['Unnamed: 8'].iloc[0]
                            days_post_injury = _tmp['Unnamed: 9'].iloc[0]
                            for i in range(len(graph.edges)):
                                path = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['pts']
                                _vals = seg_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                                _vals_0001 = seg_0001_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                                _nrn_dst_vals = nrn_dst[path[::-1,0],path[::-1,1],path[::-1,2]]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii'] = np.mean(_vals)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_std'] = np.std(_vals)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_0001'] = np.mean(_vals_0001)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['radii_0001_std'] = np.std(_vals_0001)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['mean_neuron_distance'] = np.mean(_nrn_dst_vals)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['neuron_distance_std'] = np.std(_nrn_dst_vals)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['neuron_distance_min'] = np.min(_nrn_dst_vals)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['delta'] = np.mean(_vals_0001) - np.mean(_vals)
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['gender'] = gender
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights'] = _vals
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['path_weights_0001'] = _vals_0001
                                #graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight'] = graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['weight']
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0z'] = path[0][0]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0y'] = path[0][1]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-0x'] = path[0][2]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1z'] = path[-1][0]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1y'] = path[-1][1]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['end-1x'] = path[-1][2]
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['img_start_depth'] = start_depth
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['mean_depth'] = np.mean(path[:,0])
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['euclidean-dst'] = np.sqrt(np.sum(np.square(path[-1]-path[0])))
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['subject'] = subj
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['treatment'] = treatment
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['age'] = age
                                graph[list(graph.edges)[i][0]][list(graph.edges)[i][1]]['days_post_injury'] = days_post_injury
                            nx.write_gpickle(graph, re.sub('preds','preds_graphs',re.sub('.pickle','_radii_shock.pickle',file)))
                            i+=1
print(i)

 19%|█▉        | 23/119 [00:23<05:30,  3.45s/it]/tmp/ipykernel_1975335/2211074957.py:74: RuntimeWarning:

invalid value encountered in sqrt

100%|██████████| 119/119 [17:22<00:00,  8.76s/it]

525


In [42]:
_graphs = [file for file in graphs if not os.path.exists(re.sub('preds','preds_graphs',re.sub('.pickle','_radii_shock.pickle',file)))]
print(len(_graphs))
print(_graphs)

77
['matt_preds/06162021_45-xyzres296_warped.pickle', 'matt_preds/20200109_6-XYZres69_warped.pickle', 'matt_preds/20200109_6-XYZres70_warped.pickle', 'matt_preds/20200109_6-XYZres71_warped.pickle', 'matt_preds/20200109_6-XYZres72_warped.pickle', 'matt_preds/20200109_6-XYZres73_warped.pickle', 'matt_preds/20200109_6-XYZres74_warped.pickle', 'matt_preds/20200109_6-XYZres75_warped.pickle', 'matt_preds/20200109_6-XYZres76_warped.pickle', 'matt_preds/20200109_6-XYZres77_warped.pickle', 'matt_preds/20200109_6-XYZres78_warped.pickle', 'matt_preds/20200109_6-XYZres79_warped.pickle', 'matt_preds/20200109_6-XYZres80_warped.pickle', 'matt_preds/20200217_11-XYZres91_warped.pickle', 'matt_preds/20200217_11-XYZres92_warped.pickle', 'matt_preds/20200217_11-XYZres93_warped.pickle', 'matt_preds/20200217_11-XYZres94_warped.pickle', 'matt_preds/20200217_11-XYZres96_warped.pickle', 'matt_preds/20200217_12-XYZres81_warped.pickle', 'matt_preds/20200217_12-XYZres82_warped.pickle', 'matt_preds/20200217_12-XYZ

# convert graphs to excel files

In [43]:
directory = Path('matt_preds_graphs')
files = directory.glob('*_warped_radii.pickle')
files = sorted([x.as_posix() for x in files])
print(len(files))
files[0]

402


'matt_preds_graphs/06162021_45-XYZres288_warped_radii.pickle'

In [44]:
for file in tqdm(files):
    graph = nx.read_gpickle(file)
    edge_df = nx.to_pandas_edgelist(graph)
    edge_df.to_excel(re.sub('.pickle','.xlsx',file))

100%|██████████| 402/402 [03:32<00:00,  1.89it/s]


In [45]:
directory = Path('matt_preds_graphs')
files = directory.glob('*_warped_radii_shock.pickle')
files = sorted([x.as_posix() for x in files])
print(len(files))
files[0]

42


'matt_preds_graphs/20200217_14-XYZres113_warped_radii_shock.pickle'

In [46]:
for file in tqdm(files):
    graph = nx.read_gpickle(file)
    edge_df = nx.to_pandas_edgelist(graph)
    edge_df.to_excel(re.sub('.pickle','.xlsx',file))

100%|██████████| 42/42 [00:19<00:00,  2.13it/s]
